## An Automated Portfolio Trading System with Feature Preprocessing and Recurrent Reinforcement Learning

### Setup

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from rrl_trading.data import Data, get_buy_and_hold_df, INDICATORS

from rrl_trading.model import (
    RRL, 
    train, 
    validation, 
)

from rrl_trading.backtest import (
    run_rrl_strategy, 
    create_backtest_dataset, 
    generate_layout, 
    generate_nrows, 
    generate_subplot_titles, 
    init_subplots, 
    populate_subplots,
    plot_cumulative_profits, 
    generate_df_barplot, 
    make_cumrets_barplot,
    plot_avg_portfolio_allocation, 
)

from rrl_trading.metrics import (
    calc_cumulative_returns, 
    calc_cumulative_profits, 
    calc_sharpe_ratio, 
)

from rrl_trading import save_to_yaml, read_yaml

In [9]:
import numpy as np 
import pandas as pd

from pickle import load, dump

In [10]:
# pip install cufflinks

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True) 

### Data

#### Settings

**Assets**

|  Ticker | Company  |
|---|---|
| XOM | Exxon Mobil Corporation |
| VZ | Verizon Communications Inc. |
| NKE | Nike, Inc. |
| AMAT | Applied Materials, Inc. |
| MCD | McDonald's Corporation |
| MSFT | Microsoft Corporation |
| AAP | Advance Auto Parts, Inc. |
| NOV | Nov, Inc. |

In [11]:
# assets in the portfolio
ASSETS = ["XOM", "VZ", "NKE", "AMAT", "MCD", "MSFT", "AAP", "NOV"]

n_assets = len(ASSETS)

# select n_assets assets randomly
assets = np.random.choice(ASSETS, size=n_assets, replace=False).tolist()

# select specific assets
# assets = ["AMAT", "MSFT", "MCD", "AAP"]

# date settings
start_date = "2009-12-31"
end_date = "2017-12-29"
window_size = 100

# technical indicators
momentum = [
    "MOM",
    "MACD",
    "MFI",
    "RSI"
]

volatility = [
    "ATR",
    "NATR"
]

cycle = [ 
    # "HTDCP",
    # "HTS",
    # "HTTMMM"      
]
# nb: cycle indicators seem to decrease the performance of the trading system

volume = [
    "CO",
    "OBV"
]

indicators = momentum + volatility + cycle + volume
indicators

# number of principal components (None means no PCA)
pca_ncp = None

# boolean indicating whether to apply discrete wavelet transformation
dwt = True

#### Instantiate `Data` object

In [12]:
data = Data(start_date, end_date, window_size, assets, indicators, pca_ncp, dwt)
data

# save data object into yaml configuration file 
# save_to_yaml(obj=data, path="./config/data.yaml")

[*********************100%***********************]  8 of 8 completed


Data(start_date='2009-12-31', end_date='2017-12-29', window_size=100, assets=['AAP', 'XOM', 'MCD', 'NOV', 'AMAT', 'NKE', 'MSFT', 'VZ'], indicators=['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'CO', 'OBV'], pca_ncp=None, discrete_wavelet=True)

In [13]:
df_plot = data.df.loc[:, data.df.columns.str.contains("Close")]

df_plot = df_plot.\
    rename(columns={col: col.split("_")[1] for col in df_plot.columns}).\
    pct_change().\
    dropna(axis=0).\
    apply(lambda x: (calc_cumulative_returns(x) - 1).mul(100), axis=0)

df_plot.iplot(title="% Cumulative returns per asset", yTitle="%", world_readable=True, asDates=True)

### `RRL` - Training and validation

In [7]:
initial_invest = 100
n_epochs = 100

#### No transaction fees 

$\delta=0$

In [8]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features) 
print(rrl) 

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_no_fees_dwt_2.pkl"

# with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=8, delta=0, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  12%|█▏        | 12/100 [00:00<00:04, 19.05it/s, sharpe_ratio=0.0669]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.00418790949083
Cumulative profits: 109.83979637938943
Training window: 2010-06-30/2010-10-14


Training in progress...:  81%|████████  | 81/100 [00:03<00:00, 22.79it/s, sharpe_ratio=0.204]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.3519170038957729
Cumulative profits: 115.28373422830634
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: 3.126885633464885
Cumulative profits: 301.4750268106067
Training window: 2017-03-31/2017-07-18


Training in progress...:  11%|█         | 11/100 [00:00<00:03, 25.70it/s, sharpe_ratio=0.167]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.6815808369152068
Cumulative profits: 338.18108133108734
Training window: 2017-08-23/2017-12-07


Training in progress...:   5%|▌         | 5/100 [00:00<00:03, 26.27it/s, sharpe_ratio=0.287]


Less than 30 observations in the validation set after preprocessing.


#### Transactions fees 

$\delta \in \{0.001, 0.003, 0.005\}$

In [144]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.001) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_10bps_dwt_2.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=8, delta=0.001, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  13%|█▎        | 13/100 [00:00<00:02, 29.94it/s, sharpe_ratio=0.0647]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.9702903450308717
Cumulative profits: 109.65969461294517
Training window: 2010-06-30/2010-10-14


Training in progress...:   6%|▌         | 6/100 [00:00<00:04, 19.46it/s, sharpe_ratio=0.102]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: -0.3002426025527469
Cumulative profits: 108.30075229843672
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: 1.31549541186185
Cumulative profits: 261.2446667198308
Training window: 2017-03-31/2017-07-18


Training in progress...:   3%|▎         | 3/100 [00:00<00:04, 22.44it/s, sharpe_ratio=0.116]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 4.009639791468711
Cumulative profits: 312.43993265011716
Training window: 2017-08-23/2017-12-07


Training in progress...:   2%|▏         | 2/100 [00:00<00:04, 21.68it/s, sharpe_ratio=0.197]


Less than 30 observations in the validation set after preprocessing.


In [145]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.003) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_30bps_dwt_2.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=8, delta=0.003, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:   6%|▌         | 6/100 [00:00<00:03, 26.28it/s, sharpe_ratio=0.0512]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.0468924239534596
Cumulative profits: 110.05907385803259
Training window: 2010-06-30/2010-10-14


Training in progress...:   5%|▌         | 5/100 [00:00<00:04, 19.98it/s, sharpe_ratio=0.111] 


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.8539258897278745
Cumulative profits: 118.06709539149173
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: 0.5381499337365419
Cumulative profits: 226.53598716109192
Training window: 2017-03-31/2017-07-18


Training in progress...:  13%|█▎        | 13/100 [00:00<00:02, 31.99it/s, sharpe_ratio=-.00805]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 1.3856706538460128
Cumulative profits: 242.07696629859566
Training window: 2017-08-23/2017-12-07


Training in progress...:  18%|█▊        | 18/100 [00:00<00:02, 28.14it/s, sharpe_ratio=0.022] 


Less than 30 observations in the validation set after preprocessing.


In [146]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.005) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_50bps_dwt_2.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=8, delta=0.005, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  21%|██        | 21/100 [00:00<00:02, 29.91it/s, sharpe_ratio=0.0577] 


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 0.888808639036826
Cumulative profits: 104.13583522928604
Training window: 2010-06-30/2010-10-14


Training in progress...: 100%|██████████| 100/100 [00:03<00:00, 31.82it/s, sharpe_ratio=0.0763]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 0.4159590012629939
Cumulative profits: 105.88129234835488
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.93758149539504
Cumulative profits: 118.9528511738124
Training window: 2017-03-31/2017-07-18


Training in progress...:   1%|          | 1/100 [00:00<00:06, 14.23it/s, sharpe_ratio=0.0109]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.871749950296272
Cumulative profits: 132.13107973031697
Training window: 2017-08-23/2017-12-07


Training in progress...:   1%|          | 1/100 [00:00<00:05, 16.65it/s, sharpe_ratio=-.00527]

Less than 30 observations in the validation set after preprocessing.


### Backtest

#### Buy and hold

In [41]:
# define initial invested amount
initial_invest = 100

# compute cumulative returns for "buy & hold" strategy
buy_and_hold = get_buy_and_hold_df(data.df)
buy_and_hold_sr = calc_sharpe_ratio(returns=buy_and_hold.returns, window_size=252)

#### RRL-PCA-DWT : Different transaction fees

In [43]:
# enter a preprocessing config ("", "pca", "dwt", "pca_dwt")
preprocessing = "dwt"

version = "2"

file_names = [f"rrl_{fees}_{preprocessing}" for fees in ("no_fees", "10bps", "30bps", "50bps")]

path = "./backup/"
results = {
    f.replace("results_", ""): load(open(f"{path}{f}_{version}.pkl", "rb"))
    for f in file_names
}

keys = [
    " ".join(key.split("_")[:-1])
    for key in results.keys()
]
values = list(results.values())

results = dict(zip(keys, values))

In [44]:
dfs_backtest = {k: create_backtest_dataset(v) for k, v in results.items()}

In [45]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold, 
    buy_and_hold_sr, 
    initial_invest, 
    layout)

fig.show()

In [153]:
# fig.write_image("imgs/cum-profits-rrl-dwt-2.png")

### Analysis per trading window for one setting

In [47]:
preprocessing = "dwt" 
fees = "10bps"
setting = f"rrl_{fees}_{preprocessing}"
version = "2"

results = load(open(f"{path}{setting}_{version}.pkl", "rb"))

df = create_backtest_dataset(results)
batch_names = np.unique(df["Batch"])

In [57]:
metric ="Cumulative returns"
setting = " ".join(setting.split("_"))

fig_title = f"{metric} per trading window | {setting}"

ncols = 4
nrows = generate_nrows(len(batch_names), ncols)

titles = generate_subplot_titles(batch_names)

fig = init_subplots(nrows, ncols, titles)
fig = populate_subplots(fig, df, batch_names, titles, ncols, y_varname=metric, initial_invest=initial_invest)

layout = generate_layout(
    fig_height=1200, 
    fig_width=1400, 
    fig_title=fig_title)
    
fig.update_layout(layout)

In [58]:
df_barplot = generate_df_barplot(df)
title = f"% Cumulative returns at the end of each trading window | {setting}"

layout = generate_layout(
    fig_height=500, 
    fig_width=900, 
    fig_title=title,
    x_axis_args={"title": {"text": "Trading window"}}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "%"}}
)
fig = make_cumrets_barplot(df_barplot, layout)

fig.show()

In [137]:
# fig.write_image("imgs/cum-rets-rrl-dwt-30bps.png")

**Portfolio allocation**

In [59]:
layout = generate_layout(
    fig_height=500, 
    fig_width=700, 
    fig_title="Average portfolio allocation over trading windows"
)

fig = plot_avg_portfolio_allocation(results, layout)

fig.show()

In [139]:
# fig.write_image("imgs/port-allocation-rrl-dwt-30bps.png")

#### Comparison between different preprocessing layers

| PCA | DWT | Preprocessing layer |
|---|---|---|
| 1 | 1 |Full|
|1 | 0 |Partial|
| 0 | 1 |Partial|
| 0 | 0 |None|

In [60]:
# enter a level of transaction fees ("no_fees", "10bps", "30bps", "50bps")
fees = "30bps"

file_names = [f"rrl_{fees}{version}" for version in ("", "_dwt", "_pca", "_pca_dwt")]

path = "./backup/"
results = {
    f: load(open(f"{path}{f}_1.pkl", "rb"))
    for f in file_names
}

keys = [
    " ".join(key.split("_"))
    for key in results.keys()
]
values = list(results.values())

results = dict(zip(keys, values))

In [61]:
dfs_backtest = {k: create_backtest_dataset(v) for k, v in results.items()}

In [62]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold, 
    buy_and_hold_sr, 
    initial_invest, 
    layout)

fig.show()

In [117]:
# fig.write_image("imgs/cum-profits-rrl-30bps.png")